# Function chaining

Function chaining is a pattern where multiple functions are called on the same executor consecutively. Using the
same `lithops.FunctionExecutor` object reference, multiple functions can be invoked. It increases the readability of the
code and means less redundancy. This means we chain multiple functions together with the same element reference. It’s
not necessary to attach the `lithops.FunctionExecutor` reference multiple times for each function call.

This patter is specially useful when the output of one invocation is the input of another invocation. In this case,
Lithops does not download the intermediate results to the local client, instead, the intermediate results are directly
read from the next function.

It currently works with the Futures API, and you can chain the `map()`, `map_reuce()`, `wait()` and `get_result()`
methods. Note that the returning value of one function must match the signature of the next function when chaining
multiple `map()` calls. View the next examples:

Getting the result from a single `map()` call:

In [1]:
import lithops


def my_func1(x):
    return x * 2


iterdata = [1, 2, 3]

fexec = lithops.FunctionExecutor()
res = fexec.map(my_func1, iterdata).get_result()
print(res)

[2, 4, 6]


Chain multiple `map()` calls and get the final result:

In [2]:
import lithops


def my_func1(x):
    return x * 2, 5


def my_func2(x, y):
    return x + y


iterdata = [1, 2, 3]

fexec = lithops.FunctionExecutor()
res = fexec.map(my_func1, iterdata).map(my_func2).get_result()
print(res)

[7, 9, 11]


There is no limit in the number of `map()` calls that can be chained:

In [3]:
def my_func1(x):
    return x + 2, 5


def my_func2(x, y):
    return x + y, 5, 2


def my_func3(x, y, z):
    return x + y + z


iterdata = [1, 2, 3]

fexec = lithops.FunctionExecutor()
res = fexec.map(my_func1, iterdata).map(my_func2).map(my_func3).get_result()
print(res)

[15, 16, 17]


Alternatively, you can pass the `futures` generated in a `map()` or `map_reduce()` call to the `iterdata` parameter with
the same effect. Not that in this case you will only get the results of the last `map()` execution. Results of
intermediate `map()`s are never downloaded:

In [4]:
def my_func1(x):
    return x + 2, 5


def my_func2(x, y):
    return x + y, 5, 2


def my_func3(x, y, z):
    return x + y + z


iterdata = [1, 2, 3]

fexec = lithops.FunctionExecutor()
futures1 = fexec.map(my_func1, iterdata)
futures2 = fexec.map(my_func2, futures1)
futures3 = fexec.map(my_func3, futures2)
final_result = fexec.get_result()

print(final_result)


[15, 16, 17]
